In [ ]:
!nvidia-smi

In [ ]:
import os
import random
import shutil
from PIL import Image

base_dir = "dataset"
image_base = os.path.join(base_dir, "images")
label_base = os.path.join(base_dir, "labels")

In [ ]:
splits = ["train", "val"]
split_ratio = 0.8

In [ ]:
all_images = [f for f in os.listdir(image_base) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(all_images)

split_index = int(len(all_images) * split_ratio)
split_data = {
    "train": all_images[:split_index],
    "val": all_images[split_index:]
}

for split in splits:
    split_image_folder = os.path.join(image_base, split)
    os.makedirs(split_image_folder, exist_ok=True)
    for image_file in split_data[split]:
        src = os.path.join(image_base, image_file)
        dst = os.path.join(split_image_folder, image_file)
        shutil.move(src, dst)

In [ ]:
import cv2
import numpy as np

image_size = 400

base_path = f"{base_dir}/images"

for split in splits:
    folder_path = os.path.join(base_path, split)

    for image_file in os.listdir(folder_path):
        if not image_file.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        image_path = os.path.join(folder_path, image_file)
        img = cv2.imread(image_path)

        if img is None:
            print(f"❌ Couldn't read: {image_path}")
            continue

        h, w = img.shape[:2]
        img_white = np.ones((image_size, image_size, 3), np.uint8) * 255

        aspect_ratio = h / w

        try:
            if aspect_ratio > 1:
                scale = image_size / h
                new_w = int(w * scale)
                resized_img = cv2.resize(img, (new_w, image_size))
                w_gap = (image_size - new_w) // 2
                img_white[:, w_gap:w_gap + new_w] = resized_img
            else:
                scale = image_size / w
                new_h = int(h * scale)
                resized_img = cv2.resize(img, (image_size, new_h))
                h_gap = (image_size - new_h) // 2
                img_white[h_gap:h_gap + new_h, :] = resized_img

            cv2.imwrite(image_path, img_white)

        except Exception as e:
            print(f"❌ Error processing {image_path}: {e}")


In [ ]:
class_names = [
    'A', 'B', 'C', '+', 'D', 'E', 'F', 'G', ',', 'H', 'I', '!', 'J',
    'K', 'L', 'M', 'N', 'O', '_', 'P', 'R', 'S', ';', 'T', 'U', '=',
    'V', 'Y', 'Z'
]
class_to_id = {name: idx for idx, name in enumerate(class_names)}

In [ ]:
for split in splits:
    image_folder = os.path.join(image_base, split)
    label_folder = os.path.join(label_base, split)
    os.makedirs(label_folder, exist_ok=True)

    for image_file in os.listdir(image_folder):
        if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            try:
                image_path = os.path.join(image_folder, image_file)
                with Image.open(image_path) as img:
                    width, height = img.size

                class_name = image_file[0]
                if class_name not in class_to_id:
                    print(f"❌ Skipping unknown class: {class_name}")
                    continue

                class_id = class_to_id[class_name]
                x_center = 0.5
                y_center = 0.5
                norm_width = 1.0
                norm_height = 1.0

                label_line = f"{class_id} {x_center} {y_center} {norm_width} {norm_height}\n"
                label_filename = os.path.splitext(image_file)[0] + ".txt"
                label_path = os.path.join(label_folder, label_filename)

                with open(label_path, "w") as f:
                    f.write(label_line)

            except Exception as e:
                print(f"❌ Failed to process {image_file}: {e}")


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

In [ ]:
model.train(
    data='dataset.yaml',
    epochs=20,
    batch=16,
    imgsz=400,
    project='tsl_project',
    name='tsl_yolo_train',
    exist_ok=True
)